## Imports

In [28]:
from bs4 import BeautifulSoup
import requests
from time import sleep
from tqdm.notebook import trange, tqdm
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import os

## Settings

In [ ]:
DIRECTORY_PATH = './pages'

## Download pages
It is not perfect and slow due to page loading too long. Manual download of invalid pages may be necessary.
To check which ones are bad, evaluate next cell after that cell. Then on desired page extend one accordion, 
inspect the page, copy whole html tag (don't view page sources, because it doesn't load links) and replace invalid data.

In [31]:
MAIN_URL = 'https://wizzair.com/en-gb/flights/'

def save_page(name, content):
    with open(f'{DIRECTORY_PATH}/{name}.html', mode='wb') as file: 
        file.write(content)

main_content = requests.get(MAIN_URL).content
save_page("Main", main_content)
sleep(np.random.uniform(1, 3))

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox") # linux
# chrome_options.add_argument("--disable-gpu") # windows
driver = webdriver.Chrome(options=chrome_options)

soup = BeautifulSoup(main_content, 'html.parser')
list_items = soup.findAll('li', {'class': 'list__item'})
for item in tqdm(list_items):
    a = item.find('a')
    link = a['href']
    country_name = a.contents[0].strip().replace(" ", "_")
    
    driver.get(link)
    
    # avoid clicking cookie popup
    sleep(np.random.uniform(19, 21))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # click accordion to show needed links
    sleep(np.random.uniform(0.5, 2))
    accordion = driver.find_element_by_css_selector("div[class='sf_colsIn container']")
    ActionChains(driver).move_to_element(accordion).click().perform()
    
    # save page
    sleep(np.random.uniform(4, 6))
    page_content = driver.page_source.encode("utf-8")
    save_page(country_name, page_content)
    sleep(np.random.uniform(1, 4))

#### Check which are invalid

In [32]:
for filename in os.listdir(DIRECTORY_PATH):
    with open(os.path.join(DIRECTORY_PATH, filename), 'r') as f:
        if "Flights from" not in f.read():
            print(f'{filename} is invalid')

.gitkeep is invalid
Main.html is invalid
